
```jsx
//getDownloadURL 임포트
import { ref, uploadString, getDownloadURL } from "@firebase/storage";

//사진 첨부 없이 텍스트만 트윗하고 싶을 때도 있으므로 기본 값을 ""로 해야한다.
//트윗할 때 텍스트만 입력시 이미지 url ""로 비워두기 위함
const [attachment, setAttachment] = useState("");

const onSubmit = async (event) => {
event.preventDefault();
let attachmentUrl = "";

//이미지 첨부하지 않고 텍스트만 올리고 싶을 때도 있기 때문에 attachment가 있을때만 아래 코드 실행
//이미지 첨부하지 않은 경우엔 attachmentUrl=""이 된다.
if (attachment !== "") {
//파일 경로 참조 만들기
const attachmentRef = ref(storageService, `${userObj.uid}/${uuidv4()}`);
//storage 참조 경로로 파일 업로드 하기
const response = await uploadString(attachmentRef, attachment, "data_url");
//storage 참조 경로에 있는 파일의 URL을 다운로드해서 attachmentUrl 변수에 넣어서 업데이트
attachmentUrl = await getDownloadURL(response.ref);
}

//트윗 오브젝트
const nweetObj = {
text: nweet,
createdAt: Date.now(),
creatorId: userObj.uid,
attachmentUrl,
};

//트윗하기 누르면 nweetObj 형태로 새로운 document 생성하여 nweets 콜렉션에 넣기
await addDoc(collection(dbService, "nweets"), nweetObj);

//state 비워서 form 비우기
setNweet("");

//파일 미리보기 img src 비워주기
setAttachment("");
};



//첨부 파일 url 넣는 state인 attachment 비워서 프리뷰 img src 없애기
const onClearAttachment = () => {
//null에서 빈 값("")으로 수정, 트윗할 때 텍스트만 입력시 이미지 url ""로 비워두기 위함
setAttachment("");
};
```


# Home.js
```jsx
import { useState, useEffect } from "react";
// firestore - https://firebase.google.com/docs/firestore/quickstart?hl=ko#web-version-9
import {
  collection,
  addDoc,
  // getDocs,
  query,
  orderBy,
  onSnapshot,
} from "firebase/firestore";
// storage - https://firebase.google.com/docs/storage/web/upload-files
import { ref, uploadString, getDownloadURL } from "firebase/storage";

// yarn add uuid - 특별식별자를 랜덤으로 생성
import { v4 as uuidv4 } from "uuid";

import { dbService, storageService } from "fbase";
import Jweet from "components/Jweet";

function Home({ userObj }) {
  const [jweet, setJweet] = useState("");
  const [jweets, setJweets] = useState([]);
  // 이미지url를 담을 state
  const [attachment, setAttachment] = useState("");

  useEffect(() => {
    // 실시간으로 데이터를 데이터베이스에서 가져오기
    const q = query(
      collection(dbService, "jweets"),
      // where('text', '==', 'hehe') // where뿐만아니라 각종 조건 이 영역에 때려부우면 됨
      orderBy("createdAt", "desc")
    );
    const unsubscribe = onSnapshot(q, (querySnapshot) => {
      const newArray = querySnapshot.docs.map((doc) => {
        return {
          id: doc.id,
          ...doc.data(),
        };
      });
      setJweets(newArray);
    });

    return () => {
      unsubscribe();
    };
  }, []);

  const onSubmit = async (event) => {
    event.preventDefault();
    let attachmentUrl = "";
    // 이미지 첨부하지 않고 텍스트만 올리고 싶을 때도 있기 때문에
    // attachment가 있을때만 아래 코드 실행
    //이미지 첨부하지 않은 경우엔 attachmentUrl=""이 된다.
    if (attachment) {
      // 파일 경로 참조 만들기
      const fileRef = ref(storageService, `${userObj.uid}/${uuidv4()}`);
      // storage 참조 경로로 파일 업로드 하기
      const response = await uploadString(fileRef, attachment, "data_url");
      // storage 참조 경로에 있는 파일의 URL을 다운로드해서 attachmentUrl 변수에 넣어서 업데이트
      attachmentUrl = await getDownloadURL(response.ref);
    }
    // jweet obj
    const jweetObj = {
      text: jweet,
      createdAt: Date.now(),
      creatorId: userObj.uid,
      attachmentUrl,
    };
    // 저장
    await addDoc(collection(dbService, "jweets"), jweetObj);
    // state 비워서 form 비우기
    setJweet("");
    // 파일 미리보기 img src 비워주기
    setAttachment("");
  };

  const onChange = (event) => {
    const {
      target: { value },
    } = event;
    setJweet(value);
  };

  const onFileChange = (event) => {
    // console.log(event.target.files);
    const {
      target: { files },
    } = event;

    const theFile = files[0];
    // FileReader API -> 검색
    const reader = new FileReader();

    // 2.파일리더에 after이벤트를 붙임
    reader.onloadend = (finishedEvent) => {
      // 3. 사진 string get
      const {
        currentTarget: { result },
      } = finishedEvent;
      // 4. state에 담기
      setAttachment(result);
    };
    // 1.파일을 읽음
    reader.readAsDataURL(theFile);
  };
  const onClickClearAttachment = () => setAttachment("");
  return (
    <>
      <form onSubmit={onSubmit}>
        <input
          type="text"
          placeholder="What's on your mind?"
          maxLength={120}
          onChange={onChange}
          value={jweet}
        />
        <input type="file" accept="image/*" onChange={onFileChange} />
        <input type="submit" value="Jweet" />
        {attachment && (
          <>
            <img
              src={attachment}
              width="50px"
              height="50px"
              alt="upload_image"
            />
            <button onClick={onClickClearAttachment}>Cancel</button>
          </>
        )}
      </form>
      <div>
        {jweets.map((jweet) => {
          return (
            <Jweet
              key={jweet.id}
              jweetObj={jweet}
              isOwner={jweet.creatorId === userObj.uid}
            />
          );
        })}
      </div>
    </>
  );
}

export default Home;

```